In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-dataset/modified_dataset.csv
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


# Loading the dataset

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
dataset = pd.read_csv('/kaggle/input/feedback-dataset/modified_dataset.csv')

/tmp/ipykernel_442/1770228850.py:1: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/kaggle/input/feedback-dataset/modified_dataset.csv')


In [6]:
dataset.head()

,sid,student_code,repaired_code,feedback,metacognitive_vector
0,1.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 2,\n 'feedback...","[3, 1, 3, 2, 1, 1, 2, 3, 3, 3, 2, 3, 2, 1, 2, 2]"
1,2.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 4,\n '...","[3, 3, 1, 3, 1, 1, 2, 2, 2, 3, 3, 3, 1, 2, 1, 3]"
2,3.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 2,\n '...","[3, 2, 2, 3, 3, 2, 2, 3, 3, 1, 3, 2, 1, 1, 2, 1]"
3,4.0,"x=eval(input(""Enter your age:""))\ny=str(input(...","y = input(""Enter your name:"")\nx = eval(input(...","[\n {\n 'line_number': 1,\n '...","[1, 1, 3, 3, 1, 3, 2, 3, 1, 1, 3, 2, 3, 1, 3, 3]"
4,5.0,"n = str(input(""Enter your name:""))\na = str(in...","n = str(input(""Enter your name:""))\na = str(in...","[\n {\n 'line_number': 3,\n '...","[2, 1, 2, 2, 3, 2, 3, 1, 1, 1, 2, 1, 1, 1, 2, 2]"


In [8]:
data = dataset.iloc[
:366]

In [9]:
len(data)

366

In [10]:
df = pd.DataFrame(data)  
batch_size = 8

# Creating the custom dataset from the Dataloader

In [11]:
# Custom Dataset class for DataLoader
class FeedbackDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return {
            "sid": row["sid"],
            "feedback": row["feedback"],
            "metacognitive_vector": row["metacognitive_vector"],
            "student_code": row["student_code"],
            "repaired_code": row["repaired_code"]
        }

In [12]:
# DataLoader
dataset = FeedbackDataset(df)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load gpt-neo model and tokenizer from Hugging Face
# model_name = "EleutherAI/gpt-neo-2.7B"  # You can replace this with "EleutherAI/gpt-j-6B" for gpt-j
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Loading the mistral-large-latest model

In [13]:
pip install mistralai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="TheBloke/Nous-Hermes-Llama2-GPTQ")

In [14]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os

In [16]:
os.environ["MISTRAL_API_KEY"] = "Qmo2CawcS9Ee0WkHOGUNzB4epsKmQel2"

In [17]:
import os
from dotenv import load_dotenv


# Load environment variables from the .env file
load_dotenv()

# Access the API key using os.getenv
api_key = os.getenv("MISTRAL_API_KEY")

In [18]:
import os
from mistralai import Mistral, UserMessage

In [19]:
client = Mistral(api_key=api_key)

In [27]:
model = "mistral-large-latest"

In [23]:
from transformers import MistralModel, AutoTokenizer

In [25]:
# model = MistralModel.from_pretrained(model_type)

In [26]:
# from mistralai.client import MistralClient
# client = Mistral(model)
# model = Mistral(model="mistral-large-latest").to(device)
#client = Mistral(device="cuda")

In [29]:
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.model.to(device)  # Load model onto GPU

AttributeError: 'str' object has no attribute 'model'

In [30]:
model

'mistral-large-latest'

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# # Set pad_token_id to eos_token_id to avoid generation warnings
# model.config.pad_token_id = model.config.eos_token_id

# Generating the metacognitive feedback in three steps

**Generating the high level metacognitive explanation feedback of the student.**

In [31]:
def generate_high_level_feedback(vector):
    high_level_prompt = f"""
    You are an AI tutor providing metacognitive feedback. Here is the student’s metacognitive vector:

    Metacognitive Vector (self-regulation): {vector}

    Using this vector, identify the student's:
    - Strengths in self-regulation
    - Areas needing improvement
    - Suggestions for developing better self-regulation

    generate a metacognitive based explanation for this student based on this 1x16vector value levels and self regulation which maps into
    1. Problem Understanding
    2. Requirement Identification
    3. Summarization
    4. Example Testing
    5. Goal Breakdown
    6. Pattern Recognition
    7. Algorithm Planning
    8. Step-by-Step Execution
    9. Continuous Monitoring
    10. Error Avoidance
    11. Intermediate Result Verification
    12. Implementation Monitoring
    13. Data Constraints
    14. Solution Accuracy
    15. Requirement Fulfillment
    16. Solution Reflection   these requirements 

    generate the student metacognitive level explanation using above data, precise and accurate:
    generate a paragraph wise more clear understading feedback


    """
    #inputs = tokenizer(high_level_prompt, return_tensors="pt")["input_ids"]
    
    # outputs = model.generate(
    #     inputs,
    #     num_return_sequences=1
        
        
    # )
    # generation_output = output[0][start_index:]
    # high_level_feedback = tokenizer.decode(generation_output, skip_special_tokens=True)
    # high_level_feedback = pipe(
    # high_level_prompt,
    # max_length=150,
    # temperature=0.7,   # Controls randomness; lower values make output more deterministic
    # num_return_sequences=1)

    #print(feedback[0]['generated_text'])

    messages = [
    {
        "role": "user",
        "content":high_level_prompt,
    },
    ]
    chat_response = client.chat.complete(
    model = model,
    messages = messages,
    )

#print(chat_response.choices[0].message.content)

    return chat_response.choices[0].message.content

**Generating the problem answer feedback considering the student metacognition explained high level feedback.**

In [32]:
def generate_specific_feedback(general_feedback, vector, student_code, correct_code, high_level_feedback):
    specific_feedback_prompt = f"""
    Based on the details below, generate feedback that helps the student improve their coding:

    General Feedback: {general_feedback}
    Metacognitive Vector (self-regulation): {vector}
    High-Level Feedback: {high_level_feedback}

    Student Code: {student_code}
    Correct Code: {correct_code}

    Provide feedback to:
    1. Identify code errors.
    2. Suggest improvements based on metacognitive strengths and weaknesses.
    3. Encourage reflective coding practices.

    consider above points and the student metacognitve vector and specially the high level feedback and generate more precise and concise metacognitve based explanation for the student answer by analysing the gap between student answer and the correct answer and getting the places that student need advancements considering their metacogntion explanation given by the high level feedback
    generate a paragraph wise more clear understading student personalized feedback
    """
    messages = [
    {
        "role": "user",
        "content":specific_feedback_prompt,
    },
    ]
    chat_response = client.chat.complete(
    model = model,
    messages = messages,
    )
    return chat_response.choices[0].message.content
    
    # inputs = tokenizer(specific_feedback_prompt, return_tensors="pt")
    # outputs = model.generate(
    #     inputs.input_ids,
    #     num_return_sequences=1

    # )
    # generation_output = output[0][start_index:]
    # specific_feedback = tokenizer.decode(generation_output, skip_special_tokens=True)
    # return specific_feedback

*Sample generation*

In [ ]:
# # Define inputs
# general_feedback = "Remember to follow the input format strictly to avoid runtime errors."
# vector = [2, 3, 1, 2, 3, 2, 1, 3, 2, 2, 1, 3, 2, 3, 1, 2]
# student_code = 'fahre = eval(input("enter the temperatur in fahrenheit")) result = ( fahre - 32 ) * 5 / 9'
# correct_code = 'fahre = eval(input("enter the temperature in fahrenheit")) result = ( fahre - 32 ) * 5 / 9'

In [ ]:
# # Generate high-level feedback
# high_level_feedback = generate_high_level_feedback(vector)
# print("High-Level Feedback:\n", high_level_feedback)

In [ ]:
# # Generate specific feedback based on high-level themes
# specific_feedback = generate_specific_feedback(general_feedback, vector, student_code, correct_code, high_level_feedback)
# print("\nSpecific Feedback:\n", specific_feedback)

**Generating the final metacognitive feedback for the student answer incorporating student metaognition.**

In [33]:
def final_specific_feedback(high_level_feedback, specific_feedback):
    final_feedback_prompt = f"""
    {specific_feedback}
    {high_level_feedback}

    Based on both feedback above, generate the most precise and personalized feedback for this student:
    so that student can solve the fix their answer according to their metacognition. 
    generate a paragraph wise more clear understading feedback without giving the student the direct fixes to the code, explain the issues of the student considering student's metacognition
    make the feedback more personalized and give the step  by step guidance to the student
    """
    
    messages = [
    {
        "role": "user",
        "content":final_feedback_prompt,
    },
    ]
    chat_response = client.chat.complete(
    model = model,
    messages = messages,
    )
    return chat_response.choices[0].message.content
    
    # inputs = tokenizer(specific_feedback_prompt, return_tensors="pt")
    # outputs = model.generate(
    #     inputs.input_ids,
    #     num_return_sequences=1

    # )
    # generation_output = output[0][start_index:]
    # specific_feedback = tokenizer.decode(generation_output, skip_special_tokens=True)
    # return specific_feedback

In [ ]:
# final_feedback = final_specific_feedback(high_level_feedback, specific_feedback)
# print("\nFinal Feedback:\n", final_feedback)

# Dataset creation process

In [34]:
def generate_metacognitive_feedback(row):
    print("generating metacognitive vector for:" , row['sid'])

    high_level_feedback = generate_high_level_feedback(row['metacognitive_vector'])
    specific_feedback = generate_specific_feedback(row['feedback'], row['metacognitive_vector'],row['student_code'], row['repaired_code'],high_level_feedback)

    final_feedback = final_specific_feedback(high_level_feedback, specific_feedback)

    return final_feedback

**Using batch processing**

In [35]:
# Function to generate batch feedback
def generate_metacognitive_feedback_batch(batch):
    final_feedback_batch = []
    for row in batch:
        print("Generating metacognitive feedback for:", row["sid"])
        high_level_feedback = generate_high_level_feedback(row["metacognitive_vector"])
        specific_feedback = generate_specific_feedback(
            row["feedback"], row["metacognitive_vector"],
            row["student_code"], row["repaired_code"],
            high_level_feedback
        )
        final_feedback = final_specific_feedback(high_level_feedback, specific_feedback)
        final_feedback_batch.append(final_feedback)
    return final_feedback_batch

In [36]:
# Process each batch and add the feedback to the DataFrame
final_feedback_list = []

In [38]:
len(final_feedback_list)

0

In [39]:
for batch in tqdm(dataloader):
    # Each batch is a dictionary of lists, so we create a list of dictionaries for batch processing
    batch_list = [{key: batch[key][i] for key in batch} for i in range(len(batch["sid"]))]
    feedback_batch = generate_metacognitive_feedback_batch(batch_list)
    final_feedback_list.extend(feedback_batch)
    print("final feedback list length" , len(final_feedback_list))

  0%|          | 0/46 [00:00<?, ?it/s]

Generating metacognitive feedback for: tensor(1., dtype=torch.float64)
Generating metacognitive feedback for: tensor(2., dtype=torch.float64)
Generating metacognitive feedback for: tensor(3., dtype=torch.float64)
Generating metacognitive feedback for: tensor(4., dtype=torch.float64)
Generating metacognitive feedback for: tensor(5., dtype=torch.float64)
Generating metacognitive feedback for: tensor(6., dtype=torch.float64)
Generating metacognitive feedback for: tensor(7., dtype=torch.float64)
Generating metacognitive feedback for: tensor(8., dtype=torch.float64)


  2%|▏         | 1/46 [10:31<7:53:59, 631.98s/it]

final feedback list length 8
Generating metacognitive feedback for: tensor(9., dtype=torch.float64)
Generating metacognitive feedback for: tensor(10., dtype=torch.float64)
Generating metacognitive feedback for: tensor(11., dtype=torch.float64)
Generating metacognitive feedback for: tensor(12., dtype=torch.float64)
Generating metacognitive feedback for: tensor(13., dtype=torch.float64)
Generating metacognitive feedback for: tensor(14., dtype=torch.float64)
Generating metacognitive feedback for: tensor(15., dtype=torch.float64)
Generating metacognitive feedback for: tensor(16., dtype=torch.float64)


  4%|▍         | 2/46 [20:42<7:34:07, 619.26s/it]

...

final feedback list length 240
Generating metacognitive feedback for: tensor(241., dtype=torch.float64)
Generating metacognitive feedback for: tensor(242., dtype=torch.float64)
Generating metacognitive feedback for: tensor(243., dtype=torch.float64)
Generating metacognitive feedback for: tensor(244., dtype=torch.float64)
Generating metacognitive feedback for: tensor(245., dtype=torch.float64)
Generating metacognitive feedback for: tensor(246., dtype=torch.float64)
Generating metacognitive feedback for: tensor(247., dtype=torch.float64)
Generating metacognitive feedback for: tensor(248., dtype=torch.float64)


 67%|██████▋   | 31/46 [6:02:35<3:08:30, 754.03s/it]

final feedback list length 248
Generating metacognitive feedback for: tensor(249., dtype=torch.float64)
Generating metacognitive feedback for: tensor(250., dtype=torch.float64)
Generating metacognitive feedback for: tensor(251., dtype=torch.float64)


 67%|██████▋   | 31/46 [6:06:14<2:57:12, 708.86s/it]


SDKError: API error occurred: Status 502
<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>


In [42]:
final_feedback_list[50]

"### Personalized Feedback on Code Errors and Improvements\n\n#### Identified Code Errors\n\nYour current implementation of the Fibonacci sequence uses a recursive approach, which, while straightforward, can be highly inefficient for larger inputs due to repeated computations. Specifically, your code:\n\n```python\ndef fibo(n):\n    if n == 1 or n == 2:\n        return 1\n```\n\nlacks the iterative computation that would make it more efficient for larger values of `n`.\n\n#### Suggested Improvements Based on Metacognitive Strengths and Weaknesses\n\nGiven your strengths in **Problem Understanding**, **Summarization**, **Example Testing**, **Data Constraints**, and **Solution Reflection**, you have a good foundation for tackling the problem. However, the areas needing improvement can be addressed to enhance your code.\n\n1. **Requirement Identification and Breakdown**: You need to break down the problem into smaller steps. Instead of using recursion, consider breaking down the Fibonacci

In [51]:
len(final_feedback_list)

248

In [46]:
final=[]

In [47]:
for batch in tqdm(dataloader):
    # Each batch is a dictionary of lists, so we create a list of dictionaries for batch processing
    batch_list = [{key: batch[key][i] for key in batch} for i in range(len(batch["sid"]))]
    feedback_batch = generate_metacognitive_feedback_batch(batch_list)
    final.extend(feedback_batch)
    print("final feedback list length" , len(final))

  0%|          | 0/46 [00:00<?, ?it/s]

Generating metacognitive feedback for: tensor(1., dtype=torch.float64)
Generating metacognitive feedback for: tensor(2., dtype=torch.float64)
Generating metacognitive feedback for: tensor(3., dtype=torch.float64)
Generating metacognitive feedback for: tensor(4., dtype=torch.float64)
Generating metacognitive feedback for: tensor(5., dtype=torch.float64)
Generating metacognitive feedback for: tensor(6., dtype=torch.float64)
Generating metacognitive feedback for: tensor(7., dtype=torch.float64)
Generating metacognitive feedback for: tensor(8., dtype=torch.float64)


  2%|▏         | 1/46 [11:18<8:28:33, 678.07s/it]

final feedback list length 8
Generating metacognitive feedback for: tensor(9., dtype=torch.float64)
Generating metacognitive feedback for: tensor(10., dtype=torch.float64)
Generating metacognitive feedback for: tensor(11., dtype=torch.float64)
Generating metacognitive feedback for: tensor(12., dtype=torch.float64)
Generating metacognitive feedback for: tensor(13., dtype=torch.float64)
Generating metacognitive feedback for: tensor(14., dtype=torch.float64)
Generating metacognitive feedback for: tensor(15., dtype=torch.float64)
Generating metacognitive feedback for: tensor(16., dtype=torch.float64)


  4%|▍         | 2/46 [22:45<8:21:10, 683.42s/it]

...

final feedback list length 40
Generating metacognitive feedback for: tensor(41., dtype=torch.float64)
Generating metacognitive feedback for: tensor(42., dtype=torch.float64)
Generating metacognitive feedback for: tensor(43., dtype=torch.float64)
Generating metacognitive feedback for: tensor(44., dtype=torch.float64)
Generating metacognitive feedback for: tensor(45., dtype=torch.float64)
Generating metacognitive feedback for: tensor(46., dtype=torch.float64)
Generating metacognitive feedback for: tensor(47., dtype=torch.float64)
Generating metacognitive feedback for: tensor(48., dtype=torch.float64)


 13%|█▎        | 6/46 [1:08:59<7:32:57, 679.44s/it]

final feedback list length 48
Generating metacognitive feedback for: tensor(49., dtype=torch.float64)
Generating metacognitive feedback for: tensor(50., dtype=torch.float64)
Generating metacognitive feedback for: tensor(51., dtype=torch.float64)
Generating metacognitive feedback for: tensor(52., dtype=torch.float64)
Generating metacognitive feedback for: tensor(53., dtype=torch.float64)
Generating metacognitive feedback for: tensor(54., dtype=torch.float64)
Generating metacognitive feedback for: tensor(55., dtype=torch.float64)
Generating metacognitive feedback for: tensor(56., dtype=torch.float64)


 13%|█▎        | 6/46 [1:19:10<8:47:48, 791.71s/it]


KeyboardInterrupt: 

In [48]:
print(df.columns)

Index(['sid', 'student_code', 'repaired_code', 'feedback',
       'metacognitive_vector'],
      dtype='object')


In [49]:
if 'metacognitive_feedback' not in df.columns:
    df['metacognitive_feedback'] = None

In [50]:
df.columns

Index(['sid', 'student_code', 'repaired_code', 'feedback',
       'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [52]:
final_feedback_list = final_feedback_list[:248]

In [55]:
len(data)

366

In [56]:
updated_data = data.iloc[:248]

In [57]:
len(updated_data)

248

In [62]:
updated_df = pd.DataFrame(updated_data)

In [65]:
if 'metacognitive_feedback' not in updated_df.columns:
    df['metacognitive_feedback'] = None

In [66]:
updated_df.head()

,sid,student_code,repaired_code,feedback,metacognitive_vector,metacognitive_feedback
0,1.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 2,\n 'feedback...","[3, 1, 3, 2, 1, 1, 2, 3, 3, 3, 2, 3, 2, 1, 2, 2]",### Personalized Feedback for Your Code\n\n###...
1,2.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 4,\n '...","[3, 3, 1, 3, 1, 1, 2, 2, 2, 3, 3, 3, 1, 2, 1, 3]",### Personalized Feedback for Improving Coding...
2,3.0,""""""" store the final answer in a variable named...",""""""" store the final answer in a variable named...","[\n {\n 'line_number': 2,\n '...","[3, 2, 2, 3, 3, 2, 2, 3, 3, 1, 3, 2, 1, 1, 2, 1]",### Personalized Feedback\n\n#### Understandin...
3,4.0,"x=eval(input(""Enter your age:""))\ny=str(input(...","y = input(""Enter your name:"")\nx = eval(input(...","[\n {\n 'line_number': 1,\n '...","[1, 1, 3, 3, 1, 3, 2, 3, 1, 1, 3, 2, 3, 1, 3, 3]",### Personalized Feedback for Improving Coding...
4,5.0,"n = str(input(""Enter your name:""))\na = str(in...","n = str(input(""Enter your name:""))\na = str(in...","[\n {\n 'line_number': 3,\n '...","[2, 1, 2, 2, 3, 2, 3, 1, 1, 1, 2, 1, 1, 1, 2, 2]",### Personalized Feedback for the Student\n\n#...


In [63]:
# Add final feedback to the DataFrame
updated_df['metacognitive_feedback'] = final_feedback_list

In [ ]:
# df['metacognitive_feedback'] = df.apply(generate_metacognitive_feedback, axis=1)

In [67]:
updated_df.to_csv('metacognitive_feedback_data.csv', index=False)

In [ ]:
df.to_csv("metacognitive_feedback_dataset.csv", index=False)

# LORA Finetuning